In [1]:
# Dependencies
import os
import numpy as np
import requests
import json
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import datetime as dt
import warnings
import random
warnings.filterwarnings('ignore')

In [2]:
#load 2019 prideroute zone file
rdata = pd.read_csv('prideroute.csv')
routedata=pd.DataFrame(rdata)
routedata

,LocationID,Borough,Zone,service_zone,prideroute
0,1,EWR,Newark Airport,EWR,NaN
1,2,Queens,Jamaica Bay,Boro Zone,NaN
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,NaN
3,4,Manhattan,Alphabet City,Yellow Zone,Y
4,5,Staten Island,Arden Heights,Boro Zone,NaN
...,...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone,NaN
261,262,Manhattan,Yorkville East,Yellow Zone,NaN
262,263,Manhattan,Yorkville West,Yellow Zone,NaN
263,264,Unknown,NV,NaN,NaN


In [3]:
#load 2019 june yellowcab trip data
data = pd.read_csv('yellow_tripdata_2019-06.csv')
tripdata=pd.DataFrame(data)

In [4]:
tripdata.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-06-01 00:55:13,2019-06-01 00:56:17,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1,2019-06-01 00:06:31,2019-06-01 00:06:52,1,0.0,1,N,262,263,2,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5
2,1,2019-06-01 00:17:05,2019-06-01 00:36:38,1,4.4,1,N,74,7,2,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0
3,1,2019-06-01 00:59:02,2019-06-01 00:59:12,0,0.8,1,N,145,145,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0
4,1,2019-06-01 00:03:25,2019-06-01 00:15:42,1,1.7,1,N,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5


In [5]:
tdata=tripdata[(pd.to_datetime(tripdata["tpep_pickup_datetime"])>"2019-06-22")&(pd.to_datetime(tripdata["tpep_pickup_datetime"])<"2019-07-01")]
tdata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
599520,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,13.0,1.0,0.5,3.46,0.00,0.3,20.76,2.5
599521,2,2019-06-24 10:58:03,2019-06-24 11:08:39,1,1.63,1,N,230,142,1,8.5,1.0,0.5,1.50,0.00,0.3,14.30,2.5
599522,2,2019-06-24 11:14:35,2019-06-24 11:18:08,1,0.80,1,N,239,238,2,4.5,1.0,0.5,0.00,0.00,0.3,8.80,2.5
692228,2,2019-06-24 22:25:00,2019-06-24 22:40:32,6,8.53,1,N,237,138,1,25.0,0.5,0.5,6.98,6.12,0.3,41.90,2.5
694129,2,2019-06-24 22:59:13,2019-06-25 06:22:59,6,19.07,2,N,141,132,1,52.0,0.0,0.5,12.28,6.12,0.3,73.70,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6941019,1,2019-06-30 23:23:03,2019-06-30 23:39:48,1,0.90,1,N,68,158,1,11.0,3.0,0.5,2.00,0.00,0.3,16.80,2.5
6941020,1,2019-06-30 23:50:22,2019-06-30 23:57:01,1,0.50,1,N,246,90,2,6.0,3.0,0.5,0.00,0.00,0.3,9.80,2.5
6941021,1,2019-06-30 23:58:32,2019-07-01 00:00:42,1,0.20,1,N,90,186,1,3.5,3.0,0.5,1.45,0.00,0.3,8.75,2.5
6941022,2,2019-06-30 23:23:10,2019-06-30 23:30:45,1,1.38,1,N,140,163,1,7.5,0.5,0.5,2.26,0.00,0.3,13.56,2.5


In [6]:
filterdata=tdata.merge(routedata,left_on="PULocationID",right_on="LocationID")
#filter pickup locations in the range of parade route
filterdata.dropna(inplace=True)

In [7]:
# URL for GET requests to retrieve taxi zone matching data
with open('taxi_zones.json') as f:
    data = json.load(f)
matchdata=json.dumps(data)

In [8]:
data.keys()

dict_keys(['meta', 'data'])

In [9]:
zonedata={}
for zone in data["data"]:
    limit=len(zone[10].split(","))
    choice=random.randint(2,limit-2)
    long,lati=zone[10].split(",")[choice].split()
    lon=float(long)
    lat=float(lati)
    locid=int(zone[8])
    zonename=zone[12]
    zonedata[locid]=(locid,zonename,lon,lat)
zonedata

{1: (1, 'Newark Airport', -74.18438099999997, 40.69587799999989),
 2: (2, 'Jamaica Bay', -73.83542479353247, 40.61477237228094),
 3: (3, 'Allerton/Pelham Gardens', -73.84833709099988, 40.85870184799994),
 4: (4, 'Alphabet City', -73.97197685099988, 40.72683035599993),
 5: (5, 'Arden Heights', -74.18180816599994, 40.56538457899992),
 6: (6, 'Arrochar/Fort Wadsworth', -74.06831810799993, 40.59278576599994),
 7: (7, 'Astoria', -73.91763192900005, 40.75385784899993),
 8: (8, 'Astoria Park', -73.92557304281429, 40.77845885141435),
 9: (9, 'Auburndale', -73.79501312, 40.74373670699995),
 24: (24, 'Bloomingdale', -73.97171200727101, 40.80493169127652),
 10: (10, 'Baisley Park', -73.77469730199986, 40.680486045999906),
 11: (11, 'Bath Beach', -74.01090145053502, 40.600715559854),
 12: (12, 'Battery Park', -74.01430157023238, 40.70063221177271),
 13: (13, 'Battery Park City', -74.01804288218334, 40.70462817432251),
 18: (18, 'Bedford Park', -73.8846776199998, 40.87823547699989),
 25: (25, 'Boer

In [10]:
zonedf=pd.DataFrame(zonedata).T
zonedf

,0,1,2,3
1,1,Newark Airport,-74.1844,40.6959
2,2,Jamaica Bay,-73.8354,40.6148
3,3,Allerton/Pelham Gardens,-73.8483,40.8587
4,4,Alphabet City,-73.972,40.7268
5,5,Arden Heights,-74.1818,40.5654
...,...,...,...,...
256,256,Williamsburg (South Side),-73.9693,40.711
259,259,Woodlawn/Wakefield,-73.8622,40.9013
260,260,Woodside,-73.9144,40.7405
261,261,World Trade Center,-74.0153,40.7085


In [11]:
zonedf.rename(columns={0:"locid",1:"zone",2:"lat",3:"lon"},inplace=True)
zonedf

,locid,zone,lat,lon
1,1,Newark Airport,-74.1844,40.6959
2,2,Jamaica Bay,-73.8354,40.6148
3,3,Allerton/Pelham Gardens,-73.8483,40.8587
4,4,Alphabet City,-73.972,40.7268
5,5,Arden Heights,-74.1818,40.5654
...,...,...,...,...
256,256,Williamsburg (South Side),-73.9693,40.711
259,259,Woodlawn/Wakefield,-73.8622,40.9013
260,260,Woodside,-73.9144,40.7405
261,261,World Trade Center,-74.0153,40.7085


In [12]:
pkjoindata=filterdata.merge(zonedf,left_on="PULocationID",right_on="locid")
pkjoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,congestion_surcharge,LocationID,Borough,Zone,service_zone,prideroute,locid,zone,lat,lon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
1,2,2019-06-22 00:00:16,2019-06-22 00:42:16,1,10.68,1,N,231,39,1,...,0.0,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
2,1,2019-06-22 00:25:43,2019-06-22 00:38:53,1,2.60,1,N,231,107,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
3,1,2019-06-22 00:46:18,2019-06-22 01:03:02,2,3.90,1,N,231,80,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
4,2,2019-06-22 00:04:09,2019-06-22 00:22:49,2,4.52,1,N,231,112,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635853,2,2019-06-30 23:04:47,2019-07-01 22:24:37,2,4.85,1,N,24,243,2,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,-73.9717,40.8049
635854,2,2019-06-30 23:03:57,2019-06-30 23:19:08,1,3.04,1,N,24,236,2,...,2.5,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,-73.9717,40.8049
635855,1,2019-06-30 23:03:26,2019-06-30 23:03:47,1,0.00,5,N,24,24,1,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,-73.9717,40.8049
635856,2,2019-06-30 23:06:41,2019-06-30 23:12:37,1,1.03,1,N,24,41,1,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,-73.9717,40.8049


In [13]:
pkjoindata.rename(columns={"zone":"pkzone","lat":"pklat","lon":"pklon"},inplace=True)
pkjoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,congestion_surcharge,LocationID,Borough,Zone,service_zone,prideroute,locid,pkzone,pklat,pklon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
1,2,2019-06-22 00:00:16,2019-06-22 00:42:16,1,10.68,1,N,231,39,1,...,0.0,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
2,1,2019-06-22 00:25:43,2019-06-22 00:38:53,1,2.60,1,N,231,107,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
3,1,2019-06-22 00:46:18,2019-06-22 01:03:02,2,3.90,1,N,231,80,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
4,2,2019-06-22 00:04:09,2019-06-22 00:22:49,2,4.52,1,N,231,112,2,...,2.5,231,Manhattan,TriBeCa/Civic Center,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635853,2,2019-06-30 23:04:47,2019-07-01 22:24:37,2,4.85,1,N,24,243,2,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,-73.9717,40.8049
635854,2,2019-06-30 23:03:57,2019-06-30 23:19:08,1,3.04,1,N,24,236,2,...,2.5,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,-73.9717,40.8049
635855,1,2019-06-30 23:03:26,2019-06-30 23:03:47,1,0.00,5,N,24,24,1,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,-73.9717,40.8049
635856,2,2019-06-30 23:06:41,2019-06-30 23:12:37,1,1.03,1,N,24,41,1,...,0.0,24,Manhattan,Bloomingdale,Yellow Zone,Y,24,Bloomingdale,-73.9717,40.8049


In [14]:
dojoindata=pkjoindata.merge(zonedf,left_on="DOLocationID",right_on="locid")
dojoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,zone,lat,lon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
1,2,2019-06-22 00:14:36,2019-06-22 00:36:08,5,3.39,1,N,231,161,2,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
2,2,2019-06-22 03:06:17,2019-06-22 03:20:26,2,6.18,1,N,231,161,2,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
3,1,2019-06-22 06:48:30,2019-06-22 06:56:51,1,3.70,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
4,2,2019-06-22 07:01:28,2019-06-22 07:11:04,1,3.57,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633777,2,2019-06-30 22:52:36,2019-06-30 23:38:32,1,15.29,1,N,90,184,1,...,Yellow Zone,Y,90,Flatiron,-74.002,40.7404,184,Pelham Bay Park,-73.8192,40.8482
633778,2,2019-06-28 22:46:47,2019-06-28 23:43:13,2,20.58,1,N,186,176,1,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,-73.9957,40.7491,176,Oakwood,-74.1346,40.5732
633779,1,2019-06-22 03:33:40,2019-06-22 04:20:36,1,18.20,1,N,246,176,1,...,Yellow Zone,Y,246,West Chelsea/Hudson Yards,-74.0095,40.7501,176,Oakwood,-74.1346,40.5732
633780,2,2019-06-26 21:44:47,2019-06-26 22:38:42,2,17.57,1,N,87,176,1,...,Yellow Zone,Y,87,Financial District North,-74.0048,40.7046,176,Oakwood,-74.1346,40.5732


In [15]:
dojoindata.rename(columns={"zone":"dozone","lat":"dolat","lon":"dolon"},inplace=True)
dojoindata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
0,2,2019-06-24 10:41:22,2019-06-24 10:56:33,1,3.04,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
1,2,2019-06-22 00:14:36,2019-06-22 00:36:08,5,3.39,1,N,231,161,2,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
2,2,2019-06-22 03:06:17,2019-06-22 03:20:26,2,6.18,1,N,231,161,2,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
3,1,2019-06-22 06:48:30,2019-06-22 06:56:51,1,3.70,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
4,2,2019-06-22 07:01:28,2019-06-22 07:11:04,1,3.57,1,N,231,161,1,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.0145,40.7203,161,Midtown Center,-73.9832,40.7561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633777,2,2019-06-30 22:52:36,2019-06-30 23:38:32,1,15.29,1,N,90,184,1,...,Yellow Zone,Y,90,Flatiron,-74.002,40.7404,184,Pelham Bay Park,-73.8192,40.8482
633778,2,2019-06-28 22:46:47,2019-06-28 23:43:13,2,20.58,1,N,186,176,1,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,-73.9957,40.7491,176,Oakwood,-74.1346,40.5732
633779,1,2019-06-22 03:33:40,2019-06-22 04:20:36,1,18.20,1,N,246,176,1,...,Yellow Zone,Y,246,West Chelsea/Hudson Yards,-74.0095,40.7501,176,Oakwood,-74.1346,40.5732
633780,2,2019-06-26 21:44:47,2019-06-26 22:38:42,2,17.57,1,N,87,176,1,...,Yellow Zone,Y,87,Financial District North,-74.0048,40.7046,176,Oakwood,-74.1346,40.5732


In [16]:
pridedata=dojoindata[(pd.to_datetime(dojoindata["tpep_pickup_datetime"])>"2019-06-30 16:00:00")&(pd.to_datetime(dojoindata["tpep_pickup_datetime"])<"2019-06-30 18:00:00")]

In [25]:
pridedata.sort_values(by=["tpep_pickup_datetime"],inplace=True)
pridedata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
814625,1,2019-06-30 16:00:03,2019-06-30 16:21:59,1,2.90,1,N,43,230,2,...,Yellow Zone,Y,43,Central Park,-73.9552,40.7994,230,Times Sq/Theatre District,-73.9809,40.7593
192061,2,2019-06-30 16:00:04,2019-06-30 16:05:15,6,0.80,1,N,137,162,2,...,Yellow Zone,Y,137,Kips Bay,-73.9759,40.7357,162,Midtown East,-73.9675,40.7602
531095,1,2019-06-30 16:00:04,2019-06-30 16:14:11,3,2.20,1,N,113,13,2,...,Yellow Zone,Y,113,Greenwich Village North,-73.9998,40.7333,13,Battery Park City,-74.0176,40.7047
74911,2,2019-06-30 16:00:04,2019-06-30 16:10:36,2,1.37,1,N,186,50,1,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,-73.9906,40.751,50,Clinton West,-74.0036,40.7638
148992,2,2019-06-30 16:00:08,2019-06-30 16:08:36,1,0.57,1,N,107,113,2,...,Yellow Zone,Y,107,Gramercy,-73.99,40.7351,113,Greenwich Village North,-73.9998,40.7333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46277,2,2019-06-30 17:59:54,2019-06-30 18:08:49,1,0.80,1,N,107,107,1,...,Yellow Zone,Y,107,Gramercy,-73.99,40.7351,107,Gramercy,-73.99,40.7351
854349,1,2019-06-30 17:59:54,2019-06-30 18:18:31,1,6.40,1,N,162,42,2,...,Yellow Zone,Y,162,Midtown East,-73.9675,40.7602,42,Central Harlem North,-73.9346,40.8286
438601,2,2019-06-30 17:59:56,2019-06-30 18:01:43,2,0.38,1,N,43,142,1,...,Yellow Zone,Y,43,Central Park,-73.9552,40.7994,142,Lincoln Square East,-73.9815,40.7683
36061,1,2019-06-30 17:59:56,2019-06-30 18:06:52,1,1.50,1,N,43,161,2,...,Yellow Zone,Y,43,Central Park,-73.9552,40.7994,161,Midtown Center,-73.9786,40.7624


In [17]:
priorwkdata=dojoindata[(pd.to_datetime(dojoindata["tpep_pickup_datetime"])>"2019-06-23 16:00:00")&(pd.to_datetime(dojoindata["tpep_pickup_datetime"])<"2019-06-23 18:00:00")]
priorwkdata.sort_values(by=["tpep_pickup_datetime"],inplace=True)
priorwkdata

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
37711,1,2019-06-23 16:00:02,2019-06-23 16:11:01,1,1.40,1,N,137,107,1,...,Yellow Zone,Y,137,Kips Bay,-73.972,40.7419,107,Gramercy,-73.9813,40.7414
71605,1,2019-06-23 16:00:02,2019-06-23 16:10:51,2,1.50,1,N,87,211,1,...,Yellow Zone,Y,87,Financial District North,-74.0048,40.7046,211,SoHo,-74.0014,40.7199
505293,1,2019-06-23 16:00:03,2019-06-23 16:02:14,1,0.40,1,N,186,164,1,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,-73.9957,40.7491,164,Midtown South,-73.9809,40.7535
140024,1,2019-06-23 16:00:04,2019-06-23 16:03:47,1,0.70,1,N,234,186,2,...,Yellow Zone,Y,234,Union Sq,-73.9873,40.7389,186,Penn Station/Madison Sq West,-73.9957,40.7491
208766,2,2019-06-23 16:00:04,2019-06-23 16:09:40,1,1.83,1,N,125,234,2,...,Yellow Zone,Y,125,Hudson Sq,-74.0054,40.7219,234,Union Sq,-73.9873,40.7389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64419,1,2019-06-23 17:59:54,2019-06-23 18:00:05,1,16.10,5,N,211,211,1,...,Yellow Zone,Y,211,SoHo,-74.0014,40.7199,211,SoHo,-74.0014,40.7199
221475,2,2019-06-23 17:59:55,2019-06-23 18:15:20,2,2.00,1,N,158,148,2,...,Yellow Zone,Y,158,Meatpacking/West Village West,-74.0105,40.7291,148,Lower East Side,-73.9948,40.7185
604607,1,2019-06-23 17:59:56,2019-06-23 18:15:22,1,2.20,1,N,158,100,1,...,Yellow Zone,Y,158,Meatpacking/West Village West,-74.0105,40.7291,100,Garment District,-73.9903,40.7566
106183,2,2019-06-23 17:59:56,2019-06-23 18:07:28,2,1.33,1,N,186,113,2,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,-73.9957,40.7491,113,Greenwich Village North,-73.9917,40.7352


In [18]:
# Create Engine
engine = create_engine("sqlite:///trip.db", pool_pre_ping=True)

In [19]:
pridedata.to_sql('pridedata', con=engine, if_exists='replace')

In [20]:
test=pd.read_sql("SELECT * FROM pridedata",engine)

In [21]:
test

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
0,722,2,2019-06-30 16:12:46,2019-06-30 16:55:17,1,4.36,1,N,231,161,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.014507,40.720266,161,Midtown Center,-73.983168,40.756140
1,723,1,2019-06-30 17:37:51,2019-06-30 17:57:47,1,6.00,1,N,231,161,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.014507,40.720266,161,Midtown Center,-73.983168,40.756140
2,724,2,2019-06-30 17:27:49,2019-06-30 18:15:18,2,5.69,1,N,231,161,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.014507,40.720266,161,Midtown Center,-73.983168,40.756140
3,725,1,2019-06-30 17:46:26,2019-06-30 18:09:56,2,6.10,1,N,231,161,...,Yellow Zone,Y,231,TriBeCa/Civic Center,-74.014507,40.720266,161,Midtown Center,-73.983168,40.756140
4,2184,2,2019-06-30 16:09:58,2019-06-30 16:34:42,1,2.56,1,N,68,161,...,Yellow Zone,Y,68,East Chelsea,-73.991633,40.754707,161,Midtown Center,-73.983168,40.756140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3927,632901,1,2019-06-30 16:21:04,2019-06-30 17:13:56,1,14.30,1,N,211,200,...,Yellow Zone,Y,211,SoHo,-74.001425,40.719937,200,Riverdale/North Riverdale/Fieldston,-73.912886,40.907587
3928,632910,1,2019-06-30 17:10:52,2019-06-30 17:57:03,1,0.00,1,N,107,200,...,Yellow Zone,Y,107,Gramercy,-73.981312,40.741351,200,Riverdale/North Riverdale/Fieldston,-73.912886,40.907587
3929,633233,2,2019-06-30 16:44:49,2019-06-30 17:14:44,1,14.05,1,N,107,31,...,Yellow Zone,Y,107,Gramercy,-73.981312,40.741351,31,Bronx Park,-73.870047,40.876671
3930,633601,2,2019-06-30 16:51:30,2019-06-30 17:43:59,1,11.69,1,N,249,192,...,Yellow Zone,Y,249,West Village,-74.006182,40.734439,192,Queensboro Hill,-73.820523,40.748357


In [22]:
priorwkdata.to_sql('priorwkdata', con=engine, if_exists='replace')

In [23]:
test2=pd.read_sql("SELECT * FROM priorwkdata",engine)

In [24]:
test2

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,service_zone,prideroute,locid_x,pkzone,pklat,pklon,locid_y,dozone,dolat,dolon
0,37711,1,2019-06-23 16:00:02,2019-06-23 16:11:01,1,1.40,1,N,137,107,...,Yellow Zone,Y,137,Kips Bay,-73.972049,40.741888,107,Gramercy,-73.981312,40.741351
1,71605,1,2019-06-23 16:00:02,2019-06-23 16:10:51,2,1.50,1,N,87,211,...,Yellow Zone,Y,87,Financial District North,-74.004841,40.704640,211,SoHo,-74.001425,40.719937
2,505293,1,2019-06-23 16:00:03,2019-06-23 16:02:14,1,0.40,1,N,186,164,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,-73.995749,40.749052,164,Midtown South,-73.980887,40.753481
3,140024,1,2019-06-23 16:00:04,2019-06-23 16:03:47,1,0.70,1,N,234,186,...,Yellow Zone,Y,234,Union Sq,-73.987302,40.738903,186,Penn Station/Madison Sq West,-73.995749,40.749052
4,208766,2,2019-06-23 16:00:04,2019-06-23 16:09:40,1,1.83,1,N,125,234,...,Yellow Zone,Y,125,Hudson Sq,-74.005352,40.721890,234,Union Sq,-73.987302,40.738903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7297,64419,1,2019-06-23 17:59:54,2019-06-23 18:00:05,1,16.10,5,N,211,211,...,Yellow Zone,Y,211,SoHo,-74.001425,40.719937,211,SoHo,-74.001425,40.719937
7298,221475,2,2019-06-23 17:59:55,2019-06-23 18:15:20,2,2.00,1,N,158,148,...,Yellow Zone,Y,158,Meatpacking/West Village West,-74.010544,40.729100,148,Lower East Side,-73.994808,40.718457
7299,604607,1,2019-06-23 17:59:56,2019-06-23 18:15:22,1,2.20,1,N,158,100,...,Yellow Zone,Y,158,Meatpacking/West Village West,-74.010544,40.729100,100,Garment District,-73.990276,40.756568
7300,106183,2,2019-06-23 17:59:56,2019-06-23 18:07:28,2,1.33,1,N,186,113,...,Yellow Zone,Y,186,Penn Station/Madison Sq West,-73.995749,40.749052,113,Greenwich Village North,-73.991715,40.735215
